In [1]:
from flask import Flask, jsonify, request

app = Flask(__name__)

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

df = pd.read_csv('test.csv')


In [2]:
df

,Path,Question,Answer,Context
0,1.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."
1,2.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."
2,3.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."
3,4.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."
4,5.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."
...,...,...,...,...
874,875.png,wine and beer are produced without whatdistill...,Wine-and-beer are produced without distillation.,Microbes especially yeast is being used from t...
875,876.png,wine is produced by fermentation of what by ye...,Wine is produced by fermentation of what by ye...,Microbes especially yeast is being used from t...
876,877.png,without what process is whisky obtainedferment...,"Without fermentation, the mango pulp is made i...",Microbes especially yeast is being used from t...
877,878.png,yeast and bacteria are used for commercial pro...,Yeast and bacteria are used for commercial pro...,Microbes are not only used for commercial and ...


In [3]:
df.head()

,Path,Question,Answer,Context
0,1.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."
1,2.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."
2,3.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."
3,4.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."
4,5.png,How many food plants are currently being used?,"About 10,000 food plants are being used.","Currently about 10,000 food plants are being u..."


In [4]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
model1 = SentenceTransformer('distilbert-base-nli-mean-tokens')
# 768 length vector 

c:\Users\Rajesh\Desktop\Project\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
import gensim
from gensim import similarities
def sim(sentence):
    # Define the sentence and the list of sentences
    sentence = sentence
    sentences = list(df['Question'])

    # Create a dictionary from the sentences
    dictionary = gensim.corpora.Dictionary([gensim.utils.simple_preprocess(s) for s in sentences])

    # Convert the sentences into vectors using TF-IDF
    tfidf_corpus = [dictionary.doc2bow(s) for s in [gensim.utils.simple_preprocess(s) for s in sentences]]
    tfidf_model = gensim.models.TfidfModel(tfidf_corpus)
    index = similarities.MatrixSimilarity(tfidf_model[tfidf_corpus], num_features=len(dictionary))

    # Convert the query sentence to TF-IDF vector
    query_vec = tfidf_model[dictionary.doc2bow(gensim.utils.simple_preprocess(sentence))]

    # Calculate similarities
    sims = index[query_vec]

    # Find the index of the highest similarity
    max_index = sims.argmax()

    # Return the sentence with the highest similarity
    return max_index

In [6]:
def similarity(question, lst):
    sentence_vector = model1.encode([question])[0]
    sentences_vectors = model1.encode(lst)

    # Compute the cosine similarity between the sentence vector and each of the sentences vectors
    from sklearn.metrics.pairwise import cosine_similarity
    similarities = [cosine_similarity([sentence_vector], [sentences_vector])[0][0] for sentences_vector in sentences_vectors]

    # Find the index of the most similar sentence
    most_similar_index = similarities.index(max(similarities))
    return most_similar_index

In [7]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

# load the tokenizer and the model
tokenizer = T5Tokenizer.from_pretrained('model')
model = T5ForConditionalGeneration.from_pretrained('model')

In [8]:
def answer(question, context):
    # Encode the question and context as input for the model
    input_text = f"Question: {question}\n\nContext: {context}"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate an output sequence from the model
    output_ids = model.generate(input_ids,max_length=100)

    # Decode the output sequence and return the answer
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True,)
    return output_text

In [9]:
def chat(question):
    input_text =  question 

    # encode the input text
    input_ids = tokenizer(input_text, return_tensors='pt').input_ids
    attention_mask = tokenizer(input_text, return_tensors='pt').attention_mask

    # generate an answer
    answer_ids = model.generate(input_ids, attention_mask=attention_mask,max_length=50)
    answer = tokenizer.decode(answer_ids[0], skip_special_tokens=True)
    return answer

In [10]:
def summarize(text):
    input_text = f"summarize the whole sentence into one line: {text} "
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    # Generate an output sequence from the model
    output_ids = model.generate(input_ids,max_length=100)

    # Decode the output sequence and return the answer
    output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True,)
    return output_text


In [11]:
def Question(question):
    # i = similarity(question,list(df['Question']))
    i = sim(question)

    ans = answer(question,df['Context'][i])
    return [ans,df['Path'][i]]

In [12]:
def Ques(question):
    i = sim(question)
    ans = answer(question,df['Context'][i])
    return ans

In [13]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
model2 = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
tokenizer2 = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")


In [14]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [15]:
# def tokenize(text):
#     doc = nlp(text)
#     c = doc.ents
#     s = text.split()
#     for i in c:
#       if str(i) in s:
#         s.remove(str(i))
#     text = " ".join(s)
#     for token in doc:
#         if token.pos_ == 'AUX' or token.pos_ == 'DET' or token.pos_ == 'PRON':
#            s.remove(str(token))

#     return s
def translate(text):
    
    d = {}
    # lst = tokenize(text)
    # for i in lst:
    #raj


    article_en = summarize(text)
    model_inputs = tokenizer2(article_en, return_tensors="pt")
    generated_tokens = model2.generate(
    **model_inputs,
    forced_bos_token_id=tokenizer2.lang_code_to_id["ta_IN"],max_new_tokens = 1000
    )
    translation = tokenizer2.batch_decode(generated_tokens, skip_special_tokens=True)
    #d[article_en] = translation[0]
    return translation[0]

In [16]:
@app.route('/api', methods=['GET'])
def api():
    d = {}
    q1 = str(request.args['text'])
    mode = q1[0]
    question = q1[1:]
    if mode == 'c' :
        answer= chat(question)
        d['answer'] = str(answer)
        d['path'] = str(answer)
        return d
    else:
# add the task prefix
        answer = Question(question)
        d['answer'] = str(answer[0])
        d['path'] = str(answer[1])
        return d
@app.route('/sum', methods=['GET'])
def sum():
    # Get the text from the query parameter
    text = str(request.args['text'])
# add the task prefix
    sum = summarize(text)
    d = {}
    d['sum'] = str(sum)
    return d

@app.route('/trans', methods=['GET'])
def trans():
    d={}
    text = str(request.args['text'])
    d['trans']=translate(text)
    return d



In [17]:
import random

In [18]:
df1 = {"introduction": [["What are some of the major components of the biological system on Earth?", "Microbes"], ["What contributes to human welfare?", "Microbes such as bacteria, fungi, protozoa, certain algae, viruses, viroids and prions are some of the major components of the biological system on Earth. Several microorganisms"], ["Where are microbes present?", "everywhere"], ["What can be grown on nutritive media to form colonies which can be visible?", "Microbes like bacteria and fungi"], ["What are some of the microbes useful to human welfare discussed here?", "bacteria and fungi"]], "Microbes in household products": [["What is LAB?", "probiotics"], ["What are probiotics called?", "lactic acid bacteria"], ["What check the growth of pathogenic microbes in the stomach and other parts of the digestive tract?", "Bacteria like Lactobacillus acidophilus, L. lactis and Streptococcus lactis commonly called lactic acid bacteria (LAB) are probiotics"], ["What bacteria grows in milk and converts it into curd?", "Lactobacilli"], ["How many Lactobacilli are present in fresh milk as a starter or inoculum?", "millions"], ["What is more nutritious than milk as it contains a number of organic acids and vitamins?", "Curd"], ["How is yogurt produced?", "bacterial fermentation of milk"], ["What is produced by bacterial fermentation of milk?", "Yogurt"], ["What is lactic acid produced as a byproduct?", "Yogurt"], ["How do microorganisms coagulate the milk protein and convert the lactose in the milk to lactic acids?", "Streptococcus thermophilus and Lactobacillus bulgaricus"], ["The flavour in what is due to acetaldehyde?", "yogurt"], ["What is a dairy product produced in a wide range of flavours, textures and textures?", "Cheese"], ["What is formed by coagulation of the milk protein, casein?", "Cheese"], ["During cheese production, milk is usually acidified and what enzyme is added?", "rennet"], ["The solids are separated and pressed to what?", "form cheese"], ["What are the solids separated and pressed to form?", "cheese"], ["What is fresh cheese common in South Asia?", "Paneer"], ["How is cottage cheese made?", "by curdling milk with lemon juice, vinegar and other edible acids"], ["What is the cause of large holes in Swiss cheese?", "production farge amount of carbon-di-oxide"], ["What bacteria produces carbon-di-oxide?", "Propionibacterium shermanii"], ["Who ferments the dough used in bread?", "Saccharomyces cerevisiae"], ["What bacteria ferments the dough used in bread making?", "Saccharomyces cerevisiae"], ["What is Saccharomyces cerevisiae?", "Baker\u2019s Yeast"], ["How does fermentation of glucose form ethyl alcohol and carbon di-oxide in bread?", "leavening of dough"], ["What makes bread porous and soft?", "ethyl alcohol evaporate"], ["Single cell protein refers to edible unicellular microorganisms like what?", "Spirulina"], ["What can be used as a substitute for protein rich foods?", "Protein extracts"], ["What is a substitute for protein rich foods?", "Protein extracts"], ["What is suitable for human consumption or animal feed?", "Protein extracts"]], "Microbes in industrial products": [["What is used to synthesize a number of products valuable to human beings?", "Microbes"], ["Where are products like beverages, antibiotics, organic acids, amino acids, vitamins, biofuels, single cell protein, enzymes, steroids, accines, pharmaceutical drugs, etc. produced?", "in industries"], ["Production on a large scale requires growing microbes in what?", "very large vessels called fermentors"], ["What are fermentors?", "bioreactor"], ["What is a fermentor?", "bioreactor"], ["How are bacteria produced?", "in very large vessels called fermentors"], ["What does the word antibiotic mean?", "against life"], ["Who discovered Streptomycin?", "Selman Waksman"], ["What is the name of the antibiotic that was discovered by Selman Waksman?", "Streptomycin"], ["Who was the first to use the term \"antibiotic\"?", "Selman Waksman"], ["What did Alexander Fleming observe growing in unwashed culture plates?", "a green mould"], ["What was the first antibiotic discovered by Alexander Fleming in 1926?", "penicillin"], ["What is the name of the fungi that produce penicillin?", "Penicillium notatum and Penicillium chrysogenum"], ["What is notatum and Penicillium chrysogenum?", "Penicillin"], ["What is bactericidal in action?", "enzyme responsible for the synthesis of cholesterol"], ["How does Penicillin inhibit the synthesis of the bacterial cell wall?", "bactericidal"], ["Who established the full potential of Penicillin as an effective antibiotic?", "Earnest Chain"], ["What did Earnest Chain and Howard Florey treat in World War II?", "wounded soldiers"], ["When was the Nobel prize awarded for the discovery of penicillin?", "1945"], ["What is Tetracycline?", "a broad spectrum bacteriostatic antibiotic"], ["What are antibiotics that limit the growth of bacteria called?", "bacteriostatic"], ["What is the first antibiotic of the spectrum bacteriostatic antibiotic group?", "Chlortetracycline"], ["Streptomycin is bactericidal against what?", "both gram positive and gram negative bacteria"], ["What is Streptomyces griseus bactericidal against?", "Mycobacterium tuberculosis"], ["What is erythromycin used for?", "Antibiotics"], ["When bacteria develop the ability to defeat the drug designed to kill or inhibit?", "Antibiotic resistance"], ["What is one of the most acute threats to public health?", "Antibiotic resistance"], ["What is accelerated by the misuse and over use of antibiotics?", "Antibiotic resistance"], ["What should antibiotics only be used when prescribed by a certified health professional?", "Antibiotic resistance"], ["When bacteria become resistant, antibiotics cannot fight against them and what happens to the bacteria?", "multiply"], ["What is preferred over broad spectrum antibiotics?", "Narrow spectrum antibiotics"], ["What is a term used to describe strains of bacteria that are resistant to the majority of antibiotics commonly used today?", "Superbug"], ["What is being used from time immemeorial for the production of beverages like wine, beer, and whisky?", "Microbes especially yeast"], ["What is one of the oldest alcoholic beverages known?", "Wine"], ["How is wine produced?", "fermentation of fruit juice by yeast"], ["What is zymology an applied science that deals with?", "the biochemical process of fermentation"], ["What is the biochemical process of fermentation called?", "Zymology"], ["What is brewer's yeast used for?", "fermenting malted cereals and fruit juices to produce various alcoholic beverages"], ["Wine and beer are produced without what?", "distillation"], ["Whisky, brandy and rum are obtained by what process?", "fermentation and distillation"], ["What is the science and study of wine and wine making?", "Oenology"], ["What is made from the fermentation of grape juice?", "Wine"], ["How many types of grape wine are there?", "two"], ["What type of grapes are used for red wine?", "black grapes"], ["What kind of wine is produced only from the juice of white or red rapes without their skin and stems?", "white wine"], ["How is beer produced?", "from germinated barley malt grain"], ["What is made from fermented sugarcane or molasses by Saccharomyces cerevisiae?", "Rum"], ["What type of mash is pathaneer made from?", "fermented"], ["What is a traditional drink obtained from fermenting sap of palms and coconut trees in South India?", "pathaneer"], ["In South India, what is the most common source of pathaner?", "tapping of unopened spadices of coconut"], ["How does the drink produce jaggery when boiling?", "pathaneer"], ["How much alcohol does toddy contain?", "4 percent"], ["How long does it take for toddy to become unpalatable?", "24 hours"], ["What is a refreshing drink that produces jaggery or palm sugar?", "pathaneer"], ["What is the major producer of ethanol?", "Saccharomyces cerevisiae"], ["What is ethanol referred to as?", "industrial alcohol"], ["How long does toddy become unpalatable?", "24 hours"], ["What bacteria are involved in ethanol production?", "Zymomonas mobilis and Sarcina ventriculi"], ["What are the principal substrates for the commercial production of industrial alcohol?", "molasses or corn, potatoes and wood wastes"], ["How is ethanol produced?", "milling a feed stock"], ["What is the name of the enzyme that breaks down starch into fermentable sugars?", "fungal amylase"], ["What is added to the feed stock to convert the sugars into ethanol?", "Yeast"], ["How much concentration of ethanol is ethanol in concentration?", "96 percent"], ["What is the concentration of ethanol?", "96 percent"], ["What are the two most common types of biofuels in use today?", "ethanol and biodiesel"], ["Ethanol is often used as what type of fuel?", "biofuel additive for gasoline"], ["What is mainly used as a biofuel additive for gasoline?", "Ethanol"], ["Microbes are not only used for commercial and industrial production of what?", "alcohol"], ["What are organic acids and enzymes?", "chemicals"], ["What is Rhizopus oryzae used for?", "fumaric acid"], ["What is Lactobacillus used for for lactic acid?", "Aspergillus niger"], ["Yeast and bacteria are used for what?", "commercial production of enzymes"], ["What is used to remove oily stains from the laundry?", "Lipases"], ["What is clarified by the use of pectinase,protease, and cellulases?", "Bottled juices"], ["Rennet can also be used to separate milk into what?", "solid curds"], ["Genetically engineered Streptococci are used as what?", "clot buster"], ["What is used to remove clots from the blood vessels of patients who have undergone myocardial infarction?", "Streptokinase"], ["Cyclosporin A is produced from what fungus?", "Trichoderma polysporum"], ["What is the name of the yeast that produces statins?", "Monascus purpureus"], ["Statins have been used to lower what?", "blood cholesterol levels"], ["What does statin inhibit?", "the enzyme responsible for the synthesis of cholesterol"], ["Recombinant human insulin has been produced using E. coli and what other organism?", "Saccharomyces cerevisiae"], ["E. coli and Saccharomyces cerevisiae are used for what purpose?", "Recombinant human insulin"]], "Microbes in sewage treatment and energy generation": [["What is the waste generated every day in cities and towns containing human excreta?", "Sewage"], ["What are pathogenic to humans and bio-degradablepollutants?", "organic matter and microbes"], ["How much water does domestic waste consist of?", "99 percent"], ["What should not be discharged directly into natural water bodies like rivers and streams?", "Sewage"], ["Where should sewage be treated before disposal?", "sewage treatment plants"], ["What is the main objective of a wastewater treatment process?", "improves the quality"], ["What is the purpose of the wastewater treatment process?", "to reduce organic and inorganic components in wastewater"], ["What are the main microorganisms in wastewater treatment?", "bacteria and some protozoa"], ["Why is wastewater treated?", "till the BOD is reduced"], ["What is the treatment of sewage to make it harmless?", "Microorganisms mainly bacteria and some protozoa"], ["Sewage contains what type of bacteria?", "pathogenic"], ["What must be destroyed in order to prevent the spread of diseases?", "Sewage contains pathogenic bacteria"], ["How many stages are usually performed in sewage treatment?", "three"], ["Primary treatment involves the physical removal of what?", "solid and particulate organic and inorganic materials"], ["How is floating debris removed from sewage?", "sequential filtration"], ["How is grit removed?", "sedimentation"], ["What form the primary sludge?", "All solids that settle"], ["What forms the effluent?", "supernatant"], ["What is taken from the primary settling tank for secondary treatment?", "effluent"], ["What does floc stand for?", "masses of bacteria associated with fungal filaments to form mesh like structures"], ["What do aerobic microbes consume while growing?", "organic matter in the effluent"], ["How does BOD work?", "Biochemical oxygen demand or Biological oxygen demand"], ["What does BOD stand for?", "Biochemical oxygen demand or Biological oxygen demand"], ["What is the term for the amount of oxygen that would be consumed if all the organic matter in one litre of water were oxidized by bacteria?", "BOD"], ["The greater the BOD of the sewage water, the greater what would happen to it?", "polluting potential"], ["What happens when the BOD of sewage water is reduced?", "no longer supports microbial growth"], ["What is activated sludge?", "sediment"], ["What is pumped back into the aeration tank to serve as the inoculum?", "activated sludge"], ["Where is the remaining major part of the sediment pumped into?", "anaerobic sludge digesters"], ["How do bacteria grow anaerobically?", "digest the bacteria and the fungi in the sludge"], ["What are sludge digesters?", "anaerobic"], ["What does the bacteria grow anaerobically digest?", "the bacteria and the fungi"], ["How do bacteria produce a mixture of gases during digestion?", "form biogas"], ["What is the final process that improves the quality of the waste water before it is reused, recycled or released into natural water bodies?", "Tertiary treatment"], ["What is an ideal disinfectant for?", "sewage"], ["What is an ideal disinfectant for wastewater?", "Microorganisms"], ["What is UV a chemicalfree process that can completely replace the existing chlorination system?", "Tertiary treatment"], ["What are two examples of chlorine-resistant microorganisms?", "Cryptosporidium and Giardia"], ["When was the National river conservation plan enacted?", "1995"], ["What are the major fresh water resources in our country?", "rivers"], ["What does the ministry for environment, forest and climate change do?", "initiated the Ganga action plan and the Yamuna action plan to save the major rivers of the country"], ["What is the purpose of capturing raw sewage flowing into the river?", "divert them for treatment"], ["When was the Ganga action plan launched?", "14th January 1986"], ["What is the main objective of the programme?", "to reduce organic and inorganic components in wastewater"], ["The Ministry for Environment, Forest and Climate Change initiated what action plan?", "Ganga"], ["What is the purpose of the Yamuna Action Plan?", "to save the major rivers of the country"], ["What is a bilateral project between the Government of India and Japan?", "The Yamuna Action Plan"], ["When was the Yamana Action plan formally launched?", "April 1993"], ["When was the microbial fuel cell formally launched?", "April 1993"], ["What is a bio-electrochemical system that drives an electric current by using bacteria?", "microbial fuel cell"], ["By using bacteria and mimicking bacterial interaction found in nature what do Microbial fuel cells work by allowing bacteria to oxidize and reduce organic molecules?", "electric current"], ["What is basically one big redox reaction in which electrons are being moved around?", "Bacterial respiration"], ["An anode and a cathode of a MFC are separated by what?", "proton exchange membrane"], ["What is a MFC composed of?", "an anode and a cathode separated by a proton exchange membrane"], ["What are the anode and cathode separated by?", "proton exchange membrane"]], "Microbes in the production of biogas (Gobar gas)": [["What is a mixture of different gases produced by the breakdown of organic matter in the absence of oxygen?", "Biogas"], ["What can be produced from raw materials such as agricultural wastes, manure, municipal waste, plant material, sewage, food waste?", "Biogas"], ["Under what condition is Biogas produced?", "anaerobic"], ["What is produced under anaerobic condition?", "Biogas"], ["What percentage of biogas consists of methane?", "63 percent"], ["Methane producing bacteria are called what?", "methanogens"], ["How is biogas devoid of smell?", "burns with a blue flame without smoke"], ["What is the common bacterium?", "Methanobacterium"], ["What is biogas devoid of?", "smell"], ["How does biogas burn without smoke?", "blue flame"], ["Where are Methanogens present?", "anaerobic sludge and rumen of cattle"], ["Which bacteria help in the breakdown of cellulose?", "Methanogens"], ["What is the common name for excreta of cattle called?", "Gobar"], ["What is generated by the anaerobic decomposition of cattle dung?", "Gobar gas"], ["How is Gobar gas generated?", "by the anaerobic decomposition of cattle dung"], ["In a biogas plant, what is done in an air tight cylindrical tank?", "anaerobic digestion"], ["What is the name of the air tight cylindrical tank?", "digester"], ["What is fed into the digester?", "slurry of dung"], ["How are bio-wastes collected?", "slurry of dung is fed into this digester"], ["Where are organic materials for digestion incorporated?", "a side opening"], ["What are the three stages of anaerobic digestion?", "solubilisation,acidogenesis and methanogenisis"], ["What is the outlet connected to to supply?", "a pipe"], ["How is the slurry drained through another outlet?", "fertilizer"], ["What is used as fertilizer?", "The slurry"], ["What is biogas used for?", "cooking and lighting"], ["Who developed the technology of biogas production in India?", "Indian Agricultural Research Institute (IARI) and Khadi and Village Industries Commission (KVIC)."]], "Microbes as bio control agents and biofertilisers": [["What has a deleterious effect on the health of human beings and pollute our environment?", "chemical insecticides and pesticides"], ["What is a method of controlling pest by use of microbes such as fungi, bacteria, viruses or natural substances derived from plants and animals?", "Biocontrol"], ["What are naturally occurring substances derived from?", "plants and animals"], ["What is the use of a microbes or other biological agents to control a specific pest called?", "biopesticide"], ["The lady bird beetle and dragonflies are useful to what?", "control aphids and mosquito larvae"], ["What is useful to control aphids and mosquito larvae?", "The lady bird beetle and dragonflies"], ["What is Bacillus thuringiensis commonly used as?", "biopesticide"], ["What is a soil dwelling bacterium that contains a toxin called?", "Bacillus thuringiensis"], ["Scientists have introduced what into plants?", "cry toxin"], ["What is the name of the genetically engineered insect resistant plants?", "Bt-cotton"], ["What does Bacillus thuringiensis produce during sporulation?", "crystal proteins called Delta-endotoxin"], ["Delta-endotoxins have specific activities against which order of insects?", "Lepidoptera, Diptera, Coleoptera and Hymenoptera"], ["When insects ingest toxin crystals, their alkaline digestive tract denatures what?", "insoluble crystals"], ["What does the alkaline digestive tract do to the insoluble crystals?", "denatures"], ["What happens when the cry toxin gets inserted into the gut cell membrance and paralyzes the digestive tract?", "The insect then stops eating and starves to death"], ["The insect stops eating and starves to death.", "gut cell membrance and paralyzes the digestive tract"], ["What are substances that destroy weeds without harming useful plants?", "Weedicides"], ["What are substances that destroy weeds without harming the useful plants?", "Weedicides"], ["What are compounds and secondary metabolites derived from microbes such as fungi, bacteria or protozoa?", "Bioweedicides"], ["When was the first bioherbicide developed?", "1981"], ["What control the growth of strangler vine in citrus crops?", "Mycoheribicide"], ["Trichoderma species are free living fungi that are very common in the root ecosystem.", "They are effective biocontrol agents for several plant pathogens"], ["What are pathogens that attack insects and other arthropods?", "Buculoviruses"], ["What is used as a biocontrol agent?", "The genus Nucleopolyhedrovirus"], ["What are Nucleopolyhedrovirus species specific and have narrow spectrum insecticidal applications?", "viruses"], ["Biofertilisers are formulation of living microorganisms that enrich what?", "nutrient quality of the soil"], ["What are the main sources of biofertilisers?", "bacteria, fungi and cyanobacteria"], ["What is a classical example for symbiotic nitrogen fixing bacteria?", "izobium"], ["Rhizobium infects the root nodules of what?", "leguminous plants"], ["What bacterium infects the root nodules of leguminous plants and fixes atmospheric nitrogen into organic forms?", "Rhizobium"], ["Azospirillum and Azotobacter are free living bacteria that fix what?", "atmospheric nitrogen"], ["What is the symbiotic association between a fungus and the roots of plants called?", "mycorrhiza"], ["What is fungus and the roots of plants called?", "mycorrhiza"], ["What does the fungal symbiont absorb from soil and transfer to the plant?", "phosphorus"], ["Plants having this association show other benefits such as resistance to what?", "root\u0002borne pathogens"], ["What do many members of the genus Glomus form?", "mycorrhiza"], ["What are prokaryotic free-living organisms which can fix nitrogen?", "Azospirillum and Azotobacter"], ["Oscillatoria, Nostoc, Anabaena, Tolypothrix are well known what?", "nitrogen fixing cyanobacteria"], ["Where do cyanobacteria multiply and fix molecular nitrogen?", "water logged paddy fields"], ["What does naphthalene acetic acid promote?", "plant growth and production"], ["How are biofertilisers commonly used in organic farming?", "enrich the nutrient quality of the soil"], ["How are biofertilisers used in organic farming methods?", "enrich the nutrient quality of the soil"], ["What is an example of an organic farming technique?", "Biofertilisers"], ["How does organic farming avoid synthetic substances?", "to maintain soil fertility and ecological balance"], ["What is one of the key features of organic farming?", "Protecting soil quality"], ["What is another key feature of organic agriculture?", "Protecting soil quality using organic materials and encouraging biological activity"], ["How is nitrogen fixation in soils done?", "legumes"], ["What are some of the methods used to control pests?", "crop rotation, biological diversity, natural predators, organic manures and suitable chemical, thermal and biological interventions"], ["What type of manures are used?", "organic"]], "Bioremediation": [["What is the use of naturally occurring or genetically engineered microorganisms to reduce or degrade pollutants called?", "bioremediation"], ["What is less expensive and more sustainable than other remediations available?", "Bioremediation"], ["In situ bioremediation (treatment of contaminated soil or water in the site) is grouped into what?", "ex situ bioremediation"], ["What is ex situ bioremediation?", "treatment of contaminated soil or water that is removed from the site and treated"], ["What do aerobic microbes degrade in the presence of oxygen?", "pollutants"], ["Pseudomonas putida is a what?", "genetically engineered microorganism"], ["What is a genetically engineered microorganism?", "Pseudomonas putida"], ["Who obtained a patent for this recombinant bacterial strain?", "Ananda Mohan Chakrabarty"], ["What can Nitrosomonas europaea digest?", "hydrocarbons in the oil spills"], ["What are two halogenated organic compounds?", "trichloroethylene and vinyl chloride"], ["What is Ideonella sakaiensis currently tried for recycling of?", "PET plastics"], ["Dechloromonas aromatica has the ability to degrade benzene anaerobically and to oxidize toluene and what else?", "xylene"], ["Phanerochaete chrysosporium exhibits strong potential for what?", "bioremediation"], ["Dehalococcoides species are responsible for what kind of bioremediation?", "anaerobic"], ["Dehalococcoides species are responsible for anaerobic bioremediation of toxic trichloroethene to what?", "non\u0002toxic ethane"], ["Pestalotiopsis microspora is a species of what fungus capable of breaking down and digesting polyurethane?", "endophytic"]]}

In [19]:
ls = []
i = -1

In [20]:
@app.route('/start', methods=['GET'])
def start():
    global ls
    if len(ls)!=0:
        
        ls = []
    id = str(request.args['text'])
    def get(id):
        ls.extend(random.sample(df1[id],10 if len(df1[id])>10 else len(df1
                                                                       [id])))
    get(id)
    return {"output": "success"}
    

In [21]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [22]:
def evaluate(a,b):
    a1 = nlp(a)
    a2 = nlp(b)
    return a1.similarity(a2)

In [23]:
# import gensim
# from gensim import similarities
# def sim(sentence):
#     # Define the sentence and the list of sentences
#     sentence = sentence
#     sentences = list(df['Question'])

#     # Create a dictionary from the sentences
#     dictionary = gensim.corpora.Dictionary([gensim.utils.simple_preprocess(s) for s in sentences])

#     # Convert the sentences into vectors using TF-IDF
#     tfidf_corpus = [dictionary.doc2bow(s) for s in [gensim.utils.simple_preprocess(s) for s in sentences]]
#     tfidf_model = gensim.models.TfidfModel(tfidf_corpus)
#     index = similarities.MatrixSimilarity(tfidf_model[tfidf_corpus], num_features=len(dictionary))

#     # Convert the query sentence to TF-IDF vector
#     query_vec = tfidf_model[dictionary.doc2bow(gensim.utils.simple_preprocess(sentence))]

#     # Calculate similarities
#     sims = index[query_vec]

#     # Find the index of the highest similarity
#     max_index = sims.argmax()

#     # Return the sentence with the highest similarity
#     return max_index

In [24]:
@app.route('/eval', methods=['GET'])
def eval():
    ans = str(request.args['text'])
    d={}
    if evaluate(ans,ls[i][1]) >=95:
        d['q'] = "Correct answer!!!"
    elif evaluate(ans,ls[i][1]) >=75:
        d['q'] = "You are close but the correct answer is \n" +Ques(ls[i][0])
    else:
        d['q'] = "Oops the answer is wrong!!! The answer is\n" +Ques(ls[i][0]) 
    return d

In [25]:
# %pip install pdfminer.six


In [26]:


import re
from pdfminer.high_level import extract_text

In [27]:
def extract_text_from_pdf(pdf_path):
    return extract_text(pdf_path)

In [28]:
@app.route('/pdf', methods=['GET'])

def pdf():

    ret={}
    pdf_path = str(request.args['text'])

    text = extract_text_from_pdf(pdf_path)


    # Replace specific Unicode characters
    text = text.replace('\u2018', '')  # Remove left single quotation mark
    text = text.replace('\u2019', '')  # Remove right single quotation mark
    text = text.replace('\u201C', '')  # Remove left double quotation mark
    text = text.replace('\u201D', '')  # Remove right double quotation mark

    # Split the text into lines
    lines = text.split('\n')

    # Initialize an empty dictionary to hold your data
    data = {}

    # Initialize empty strings to hold the current heading and current content
    current_heading = ''
    current_content = ''

    # Define patterns for unwanted text
    unwanted_patterns = [
        r'^\s*\[Page [0-9]+\]',  # Remove lines containing page numbers like "[Page 1]"
    ]

    # Iterate over each line
    for line in lines:
        # Apply unwanted text filtering
        if any(re.match(pattern, line) for pattern in unwanted_patterns):
            continue

        # If the line matches the pattern of a heading (this will depend on your specific textbook)
        if re.match(r'^[0-9]+\.[0-9]+ ', line):  # Example pattern: "1.1 "
            # This line is a heading, so update current_heading and initialize a new key-value pair in data
            # If there was a previous heading, save its content
            if current_heading:
                data[current_heading] = current_content.strip()

            # Update current_heading and reset current_content
            current_heading = line.strip()
            current_content = ''
        elif re.match(r'^[0-9]+\.[0-9]+\.[0-9]+ ', line):  # Example pattern: "1.1.1 "
            # This line is a heading, so update current_heading and initialize a new key-value pair in data
            # If there was a previous heading, save its content
            if current_heading:
                data[current_heading] = current_content.strip()

            # Update current_heading and reset current_content
            current_heading = line.strip()
            current_content = ''
        elif current_heading:
            # This line is not a heading, so it's part of the current section's text. Append it to the current section's content.
            current_content += ' ' + line.strip()

    # If there's content left after the loop ends, save it for the last heading
    if current_heading:
        data[current_heading] = current_content.strip()

    # Write data to JSON file
    # with open('output.json', 'w') as json_file:
    #     json.dump(data, json_file, indent=2)

    filtered_data = {k: v for k, v in data.items() if v is not None and v != ''}


    ret['out'] = filtered_data
    print("Output saved to 'outjson.json'")
    return ret

In [29]:
from pptx import Presentation
import json
import re

In [30]:
def extract_text_from_shape(shape):
    if shape.has_text_frame:
        text_parts = []
        for paragraph in shape.text_frame.paragraphs:
            for run in paragraph.runs:
                text_parts.append(run.text.strip())
        return ' '.join(text_parts)
    return ""

def extract_text_from_slide(slide):
    text = []
    for shape in slide.shapes:
        text_part = extract_text_from_shape(shape)
        if text_part:
            text.append(text_part)
    return ' '.join(text)


In [31]:
@app.route('/ppt', methods=['GET'])

def ppt():
    ret={}
    path2 = str(request.args['text'])
    presentation = Presentation(path2)

    material = {}

    current_topic = None

    for slide in presentation.slides:
        slide_info = {}
        slide_text = extract_text_from_slide(slide)
        if slide.shapes.title is not None:
            current_topic = slide.shapes.title.text.replace('\u000b', '').strip()
        if current_topic not in material:
            material[current_topic] = ""
        if slide_text.strip():
            # Check if the slide text contains the topic and remove it
            slide_text = slide_text.replace(current_topic, "").strip()
            material[current_topic] += " " + slide_text.strip()

    # Clean up the text to form perfect sentences
    for topic, content in material.items():
        # Remove extra spaces and tabs
        content = re.sub(r'\s+', ' ', content)
        # Remove leading/trailing spaces and dots
        content = content.strip('. ')
        material[topic] = content

    # with open('outjson3.json', 'w') as json_file:
    #     json.dump(material, json_file, indent=4)
    # ans =  jsonify(material)
    ret['out'] = material
    print("Output saved to 'outjson.json'")
    return ret


In [32]:
@app.route('/ask', methods=['GET'])

def ask():
    global i 
    i = (i+1)%10
    d= {}
    d['q'] = ls[i][0]
    return d

In [33]:
@app.route('/img', methods=['GET'])
def img():
    paragraph =  str(request.args['text'])
    doc = nlp(paragraph)
    d = {}
    l=[]
    sentences = [sent.text for sent in doc.sents]
    for sentence in sentences:
        # Your data generation logic
        # i = similarity(sentence, list(df['Answer']))
        i = sim(sentence)

        l.append(df['Path'][i])
    d['img'] = l
    return d


In [34]:

if  __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [01/Jun/2024 12:47:03] "GET /trans?text=Microbes%20such%20as%20bacteria,%20fungi,%20protozoa,%20certain%20algae,%20viruses,%20viroids%20and%20prions%20are%20some%20of%20the%20major%20components%20of%20the%20biological%20system%20on%20Earth.%20Several%20microorganisms%20arebeneficial%20and%20contribute%20to%20human%20welfare.%20Microbes%20are%20presenteverywhere%20–%20in%20soil,%20water,%20air%20and%20within%20bodies%20of%20animals%20and%20plants.%20Microbes%20like%20bacteria%20and%20fungi%20can%20be%20grown%20on%20nutritive%20media%20to%20form%20colonies%20which%20can%20be%20visibly%20seen.%20Some%20of%20the%20microbes%20useful%20to%20human%20welfare%20are%20discussed%20here HTTP/1.1" 200 -
127.0.0.1 - - [01/Jun/2024 14:20:24] "GET /trans?text=Microbes%20such%20as%20bacteria,%20fungi,%20protozoa,%20certain%20algae,%20viruses,%20viroids%20and%20prions%20are%20some%20of%20the%20major%20components%20of%20the%20biologic

Output saved to 'outjson.json'


127.0.0.1 - - [01/Jun/2024 14:37:24] "GET /trans?text=Little%20Millet%20Botanical%20%20name-%20Panicum%20sumatrense%20%20This%20is%20one%20of%20the%20oldest%20millets%20and%20is%20native%20to%20India.%20The%20species%20name%20is%20based%20on%20a%20specimen%20collected%20from%20Sumatra.%20It%20is%20rich%20in%20iron%20and%20fibre%20than%20rice%20that%20makes%20it%20best%20for%20diabetes.%20%20Uses%20%20It%20is%20cooked%20like%20rice%20and%20also%20milled%20and%20baked.%20It%20%20cures%20%20anaemic%20%20condition,%20%20constipation%20%20and%20other%20gastrological%20problems.%20%20Foxtail%20Millet%20Botanical%20name%20:%20Setaria%20italica%20%20is%20%20one%20%20of%20%20This%20the%20%20oldest%20%20millet%20%20used%20traditionally%20%20in%20India.%20Which%20is%20domesticated%20first%20in%20China%20about%206000%20years.%20Rich%20in%20protein,%20carbohydrate,%20vitamin%20B%20and%20C,%20Potassium%20and%20Calcium.%20%20Uses%20It%20%20supports%20%20in%20%20strengthening%20%20of%20%20heart%20%20a

In [ ]:
import cv2
import os
import time
import dlib
from scipy.spatial import distance
import mediapipe as mp
from open_cv import HandTrackingModule  as htm 

mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

tipIds = [4, 8, 12, 16, 20]



def calculate_EAR(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear_aspect_ratio = (A + B) / (2.0 * C)
    return ear_aspect_ratio

def get_className(classNo):
    p = []
    for i in os.listdir(r'C:\Users\Rajesh\Desktop\Project\open_cv\images'):
        p.append(i)
    return p[classNo]

doubt=0
dict1={}
for i in os.listdir(r'C:\Users\Rajesh\Desktop\Project\open_cv\images'):
    dict1[i]=0


wCam, hCam = 640, 480
cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
font = cv2.FONT_HERSHEY_COMPLEX

facedetect = cv2.CascadeClassifier(r"C:\Users\Rajesh\Desktop\Project\open_cv\haar_face.xml")

model = cv2.face.LBPHFaceRecognizer_create()
model.read(r'C:\Users\Rajesh\Desktop\Project\open_cv\face_trained2.yml')

hog_face_detector = dlib.get_frontal_face_detector()
dlib_facelandmark = dlib.shape_predictor(r"C:\Users\Rajesh\Desktop\Project\open_cv\shape_predictor_68_face_landmarks.dat")


In [ ]:
# New

from flask import Flask, render_template
from flask_socketio import SocketIO
from flask_cors import CORS  # Import the CORS module
import random

app = Flask(__name__)
CORS(app, origins=["http://localhost:58015"])  # Enable CORS for your Flask app
socketio = SocketIO(app)

# ... rest of your code ...




@app.route('/')
def index():
    return render_template('index.html')

def data_generator():
    global dict1 
    global doubt
    while True:
        finger_counts = []
        drowsy_list = []
        success, imgOrignal = cap.read()
        img = cv2.cvtColor(imgOrignal, cv2.COLOR_BGR2RGB)

        # Face Recognition
        faces = facedetect.detectMultiScale(imgOrignal, 1.3,5)
        for x, y, w, h in faces:
            crop_img = imgOrignal[y:y + h, x:x + h]
            #img = cv2.resize(crop_img, (224, 224))
            #img = img.reshape(1, 224, 224, 3)
            crop_img = cv2.cvtColor(crop_img,cv2.COLOR_BGR2GRAY)
            cropped = (imgOrignal[:, x-h:x + h + h//2]).copy()

            classIndex, probabilityValue = model.predict(crop_img)
            #print(probabilityValue, get_className(classIndex))
            if classIndex >= 0:
                cv2.rectangle(imgOrignal, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.rectangle(imgOrignal, (x, y - 40), (x + w, y), (0, 255, 0), -2)
                cv2.putText(imgOrignal, str(get_className(classIndex)), (x, y - 10), font, 0.75, (0, 0, 255), 1,
                            cv2.LINE_AA)

                    # Drowsy Detection
                    #gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
                faces = hog_face_detector(crop_img)
                for face in faces:
                    face_landmarks = dlib_facelandmark(crop_img, face)
                    leftEye = []
                    rightEye = []

                    for n in range(36, 42):
                        x = face_landmarks.part(n).x
                        y = face_landmarks.part(n).y
                        leftEye.append((x, y))
                        next_point = n + 1
                        if n == 41:
                            next_point = 36
                        x2 = face_landmarks.part(next_point).x
                        y2 = face_landmarks.part(next_point).y
                        cv2.line(crop_img, (x, y), (x2, y2), (0, 255, 0), 1)

                    for n in range(42, 48):
                        x = face_landmarks.part(n).x
                        y = face_landmarks.part(n).y
                        rightEye.append((x, y))
                        next_point = n + 1
                        if n == 47:
                            next_point = 42
                        x2 = face_landmarks.part(next_point).x
                        y2 = face_landmarks.part(next_point).y
                        cv2.line(crop_img, (x, y), (x2, y2), (0, 255, 0), 1)

                    left_ear = calculate_EAR(leftEye)
                    right_ear = calculate_EAR(rightEye)

                    EAR = (left_ear + right_ear) / 2
                    EAR = round(EAR, 2)

                    if EAR < 0.19:
                        cv2.putText(crop_img, "DROWSY", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 4)
                        cv2.putText(crop_img, "Are you Sleepy?", (20, 400), cv2.FONT_HERSHEY_SIMPLEX, 2,
                                    (0, 0, 255), 4)
                        drowsy_list.append(get_className(classIndex))

            cv2.imshow("Cropped",cropped)
            cv2.imshow("Drowsy", crop_img)
        cv2.imshow("Face Recognition", imgOrignal)
        
        for j in drowsy_list :
            (dict1[j]) += 1

        for i,j in dict1.items():
            if j > 3 :
                socketio.emit('data', {'data': i}, namespace='/drowsy')
                socketio.sleep(1)
                dict1[i] =0  

        key = cv2.waitKey(1)
        if key == 27:
            break
        

    cap.release()
    cv2.destroyAllWindows()        
            
@socketio.on('connect', namespace='/drowsy')
def handle_connect():
    socketio.start_background_task(data_generator)

In [ ]:
# # Old with Fin_Count

# from flask import Flask, render_template
# from flask_socketio import SocketIO
# from flask_cors import CORS  # Import the CORS module
# import random

# app = Flask(__name__)
# CORS(app, origins=["http://localhost:58015"])  # Enable CORS for your Flask app
# socketio = SocketIO(app)

# # ... rest of your code ...




# @app.route('/')
# def index():
#     return render_template('index.html')

# def data_generator():
#     global dict1 
#     global doubt
#     while True:
#         finger_counts = []
#         drowsy_list = []
#         success, imgOrignal = cap.read()
#         img = cv2.cvtColor(imgOrignal, cv2.COLOR_BGR2RGB)

#         # Face Recognition
#         faces = facedetect.detectMultiScale(imgOrignal, 1.3,5)
#         for x, y, w, h in faces:
#             crop_img = imgOrignal[y:y + h, x:x + h]
#             #img = cv2.resize(crop_img, (224, 224))
#             #img = img.reshape(1, 224, 224, 3)
#             crop_img = cv2.cvtColor(crop_img,cv2.COLOR_BGR2GRAY)
#             cropped = (imgOrignal[:, x-h:x + h + h//2]).copy()

#             classIndex, probabilityValue = model.predict(crop_img)
#             #print(probabilityValue, get_className(classIndex))
#             if classIndex >= 0:
#                 cv2.rectangle(imgOrignal, (x, y), (x + w, y + h), (0, 255, 0), 2)
#                 cv2.rectangle(imgOrignal, (x, y - 40), (x + w, y), (0, 255, 0), -2)
#                 cv2.putText(imgOrignal, str(get_className(classIndex)), (x, y - 10), font, 0.75, (0, 0, 255), 1,
#                             cv2.LINE_AA)

#                 # Landmarks Detection
#                 with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
#                     try : 
#                         image = cv2.cvtColor(cropped, cv2.COLOR_BGR2RGB)
#                         results = holistic.process(image)
#                     except:
#                         continue
                    
#                     '''mp_drawing.draw_landmarks(imgOrignal, results.face_landmarks, mp_holistic.FACE_CONNECTIONS,
#                                             mp_drawing.DrawingSpec(color=(80, 110, 10), thickness=1,
#                                                                     circle_radius=1),
#                                             mp_drawing.DrawingSpec(color=(80, 256, 121), thickness=1,
#                                                                     circle_radius=1))'''

#                     mp_drawing.draw_landmarks(cropped, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
#                                             mp_drawing.DrawingSpec(color=(80, 22, 10), thickness=2, circle_radius=4),
#                                             mp_drawing.DrawingSpec(color=(80, 44, 121), thickness=2, circle_radius=2))
#                     #print("right",results.right_hand_landmarks)

#                     mp_drawing.draw_landmarks(cropped, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
#                                             mp_drawing.DrawingSpec(color=(121, 22, 76), thickness=2, circle_radius=4),
#                                             mp_drawing.DrawingSpec(color=(121, 44, 250), thickness=2, circle_radius=2))

#                     mp_drawing.draw_landmarks(cropped, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
#                                             mp_drawing.DrawingSpec(color=(245, 117, 66), thickness=2, circle_radius=4),
#                                             mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

#                     # Finger Counting
#                     detector = htm.handDetector()
#                     lmList=[]
#                     if results.right_hand_landmarks :
#                         #img = detector.findHands(imgOrignal)
#                         lmList = detector.findPosition(img=cropped , landmarks=[results.right_hand_landmarks])
#                         #print("lmList", lmList , "right",results.right_hand_landmarks)
#                     elif results.left_hand_landmarks :
#                         #img = detector.findHands(imgOrignal)
#                         lmList = detector.findPosition(img=cropped, landmarks=[results.left_hand_landmarks])
#                         #print("lmList", lmList , "right",results.right_hand_landmarks)


#                     #cropped = detector.findHands(cropped)
#                     #lmList = detector.findPosition(img=cropped, draw=False)
#                     if len(lmList) != 0:
#                         fingers = []
#                         if lmList[4][1] > lmList[3][1]:
#                             fingers.append(1)
#                         else:
#                             fingers.append(0)

#                         for id in range(1, 5):
#                             if lmList[tipIds[id]][2] < lmList[tipIds[id] - 2][2]:
#                                 fingers.append(1)
#                             else:
#                                 fingers.append(0)

#                         totalFingers = fingers.count(1)
#                         if totalFingers==5 :
#                             doubt+=1
#                         finger_counts.append((get_className(classIndex), totalFingers))
#                         #h, w, c = overlayList[totalFingers - 1].shape
#                         #imgOrignal[y:y + h, x:x + w] = overlayList[totalFingers - 1]

#                     # Drowsy Detection
#                     #gray = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
#                     faces = hog_face_detector(crop_img)
#                     for face in faces:
#                         face_landmarks = dlib_facelandmark(crop_img, face)
#                         leftEye = []
#                         rightEye = []

#                         for n in range(36, 42):
#                             x = face_landmarks.part(n).x
#                             y = face_landmarks.part(n).y
#                             leftEye.append((x, y))
#                             next_point = n + 1
#                             if n == 41:
#                                 next_point = 36
#                             x2 = face_landmarks.part(next_point).x
#                             y2 = face_landmarks.part(next_point).y
#                             cv2.line(crop_img, (x, y), (x2, y2), (0, 255, 0), 1)

#                         for n in range(42, 48):
#                             x = face_landmarks.part(n).x
#                             y = face_landmarks.part(n).y
#                             rightEye.append((x, y))
#                             next_point = n + 1
#                             if n == 47:
#                                 next_point = 42
#                             x2 = face_landmarks.part(next_point).x
#                             y2 = face_landmarks.part(next_point).y
#                             cv2.line(crop_img, (x, y), (x2, y2), (0, 255, 0), 1)

#                         left_ear = calculate_EAR(leftEye)
#                         right_ear = calculate_EAR(rightEye)

#                         EAR = (left_ear + right_ear) / 2
#                         EAR = round(EAR, 2)

#                         if EAR < 0.19:
#                             cv2.putText(crop_img, "DROWSY", (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 3, (0, 0, 255), 4)
#                             cv2.putText(crop_img, "Are you Sleepy?", (20, 400), cv2.FONT_HERSHEY_SIMPLEX, 2,
#                                         (0, 0, 255), 4)
#                             drowsy_list.append(get_className(classIndex))

#             cv2.imshow("Cropped",cropped)
#             cv2.imshow("Drowsy", crop_img)
#         cv2.imshow("Face Recognition", imgOrignal)
        
#         # if finger_counts != [] :
#         #     socketio.emit('data', {'data': finger_counts}, namespace='/fingers')

#         # print()
#         # print("Drowsy List:", drowsy_list)
         

#         # if doubt >= 5 :
#         #     socketio.emit('data', {'data': "Doubt"}, namespace='/doubt')
#         #     doubt=0

#         for j in drowsy_list :
#             (dict1[j]) += 1

#         for i,j in dict1.items():
#             if j > 1 :
#                 socketio.emit('data', {'data': i}, namespace='/drowsy')
#                 socketio.sleep(1)
#                 dict1[i] =0  

#         key = cv2.waitKey(1)
#         if key == 27:
#             break
        

#     cap.release()
#     cv2.destroyAllWindows()        
            
# @socketio.on('connect', namespace='/drowsy')
# def handle_connect():
#     socketio.start_background_task(data_generator)

In [ ]:
# cap.release()
# cv2.destroyAllWindows()

NameError: name 'cap' is not defined

In [ ]:
if __name__ == "__main__":
    socketio.run(app,port=5001,allow_unsafe_werkzeug=True)

: 

: 